# Learn

In [2]:
from typing import Literal
from langgraph.types import interrupt, Command
from typing import TypedDict
import uuid
from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command

class State(TypedDict):
    llm_output: int



async def human_approval(state: State) -> Command[Literal["human_node_1", "human_node_2"]]:
    is_approved = True
    llm_output = 0
    print("---------------大纲生成节点---------------")
    print(is_approved)
    print("---------------大纲生成节点---------------")
    while is_approved:
        llm_output = state["llm_output"] + 1
        is_approved = interrupt(
            {
                "question": "是否重新开始？",
                # Surface the output that should be
                # reviewed and approved by the human.
                "llm_output": state["llm_output"]
            }
        )
        print("---------------中断完成---------------")
        print(is_approved)
        print("---------------中断完成---------------")
    return Command(goto="human_node_1")


async def human_node_1(state: State):
    value = interrupt({"text_to_revise": state["llm_output"]})
    return {"llm_output": value}


async def human_node_2(state: State):
    value = interrupt({"text_to_revise": state["llm_output"]})
    return {"llm_output": value}


checkpointer = InMemorySaver()
graph_builder = StateGraph(State)
graph_builder.add_edge(START, "human_approval")
graph_builder.add_node("human_approval", human_approval)
graph_builder.add_node("human_node_1", human_node_1)
graph_builder.add_node("human_node_2", human_node_2)
graph_builder.add_edge("human_node_1", END)
graph_builder.add_edge("human_node_2", END)
graph = graph_builder.compile(checkpointer=checkpointer)

# After running the graph and hitting the interrupt, the graph will pause.
# Resume it with either an approval or rejection.
thread_config = {"configurable": {"thread_id": "7"}}
async for chunk in graph.astream({"llm_output": 0}, config=thread_config,stream_mode=["custom", "updates"]):
    print(chunk)
print("---------------用户同意---------------")
async for chunk in graph.astream(Command(resume=False), config=thread_config,stream_mode=["custom", "updates"]):
    print(chunk)

---------------大纲生成节点---------------
True
---------------大纲生成节点---------------
('updates', {'__interrupt__': (Interrupt(value={'question': '是否重新开始？', 'llm_output': 0}, id='4b8f8e6397d1ca73abe9b04429447bdb'),)})
---------------用户同意---------------
---------------大纲生成节点---------------
True
---------------大纲生成节点---------------
---------------中断完成---------------
False
---------------中断完成---------------
('updates', {'human_approval': None})
('updates', {'__interrupt__': (Interrupt(value={'text_to_revise': 0}, id='1e9217035bc65f6fd7341f2ceaa6e6e5'),)})


---------------大纲生成节点---------------
1
---------------大纲生成节点---------------
---------------中断完成---------------
True
---------------中断完成---------------


{'llm_output': 0,
 '__interrupt__': [Interrupt(value={'question': '是否重新开始？', 'llm_output': 0}, id='cc1af7b8ec4941cda5da6b553ee6e141')]}

---------------大纲生成节点---------------
1
---------------大纲生成节点---------------
---------------中断完成---------------
True
---------------中断完成---------------


{'llm_output': 0,
 '__interrupt__': [Interrupt(value={'question': '是否重新开始？', 'llm_output': 0}, id='cc1af7b8ec4941cda5da6b553ee6e141')]}

# Backup

In [ ]:
import asyncio
import time
from typing import Dict, Any, cast
from graph import create_writing_assistant_graph
from tools import load_knowledge_bases
from langgraph.types import Command

initial_state = {
    "topic": "Python异步编程最佳实践",
    "user_id": "complete_streaming_test",
    "max_words": 600,
    "style": "technical",
    "language": "zh"
}
config: Dict[str, Any] = {"configurable": {"thread_id": "complete_streaming_004"}}


# 生成大纲
graph = create_writing_assistant_graph()
async for chunk in graph.astream(initial_state, cast(Any, config), stream_mode=["custom", "updates"]):
    print(chunk)

In [ ]:
# 确认大纲
async for chunk in graph.astream(Command(goto="rag_enhancement"), cast(Any, config), stream_mode=["custom", "updates"]):
    print(chunk)

In [ ]:
# 确认搜索知识库
async for chunk in graph.astream(Command(resume="yes"), cast(Any, config), stream_mode=["custom", "updates"]):
    print(chunk)

In [ ]:
# 确认联网搜索并开始生成
async for chunk in graph.astream(Command(resume="yes"), cast(Any, config), stream_mode=["custom", "updates"]):
    print(chunk)

---------------大纲生成节点---------------
1
---------------大纲生成节点---------------
---------------中断完成---------------
True
---------------中断完成---------------


{'llm_output': 0,
 '__interrupt__': [Interrupt(value={'question': '是否重新开始？', 'llm_output': 0}, id='cc1af7b8ec4941cda5da6b553ee6e141')]}